### Importar librerías

In [ ]:
import numpy as np
import pandas as pd
import datetime
import re

# importar fichero con utilidades propias
from commons import myfunctions as myfunc

start_time = datetime.datetime.now()

### Variables globales

In [ ]:
myfunc.reset_vars()
myfunc.NOTEBK_FILENAME = myfunc.get_nb_name()

myfunc.check_enviroment(myfunc.DATA_DIR, myfunc.CFDNA_DIR, myfunc.GENCODE_DIR, myfunc.H5_DIR, myfunc.LOG_DIR, myfunc.CSV_DIR, myfunc.MODEL_DIR, myfunc.EXEC_DIR, myfunc.MET_DIR)

myfunc.print_vars()

### 1.- Unión de los ficheros con los distintos tipos de cáncer en uno.

In [ ]:
myfunc.verbose("Inicio de unión de ficheros")

# dataframe con columnas/características
df_r = pd.DataFrame()

# dataframe con etiquetas/target/tipos de cáncer
df_c = pd.DataFrame()

unir_columnas=["id", "Chr", "Start", "End", "Strand", "Length"]

for target1 in myfunc.TARGET_MAPPING:
    myfunc.verbose(f"Tratando el fichero '{target1}'")

    fichero1='GSE202988_5hmC_ReadCount_BinnedGenome_'+target1+'.txt'
    df_tmp=myfunc.read_data_file(fichero1, myfunc.CFDNA_DIR, myfunc.N_ROWS)

    df_c=myfunc.add_target_row(df_tmp, df_c, target1, unir_columnas)

    if df_r.empty:
        df_r = df_tmp.copy()
    else:
        df_r = df_r.merge(df_tmp, on=unir_columnas, how="outer")
    
    myfunc.verbose(f"Antes de unir df_tmp : (row,cols) -> {df_tmp.shape}")
    myfunc.verbose(f"Después de unir df_r : (row,cols) -> {df_r.shape}")
    myfunc.verbose(f"Después de unir df_c : (row,cols) -> {df_c.shape}")

myfunc.verbose("Fin unión de ficheros")

Visualizar el dataframe, eliminar muestra atipica Z82 y guardarlo en un fichero h5

In [ ]:
display(df_r)
display(df_r.shape)
df_r.drop(["Z82"], axis=1, inplace=True)
display(df_r.shape)

myfunc.save_df_to_h5(df_r, "files_joined", myfunc.H5_DIR)


Visualizar el dataframe de la clase, eliminar la muestra Z82

In [ ]:
display(df_c)
display(df_c["Z82"])
display(df_c.shape)
df_c.drop("Z82", axis=1, inplace=True)
display(df_c.shape)


Trasponer el dataframe de clases para tener asociado el id de la muestra al tipo de cáncer.

Ver información del contenido del dataframe y guardarlo en un fichero.

In [ ]:
quitar_columnas = ["Chr", "Start", "End", "Strand", "Length"]
df_c.drop(quitar_columnas, axis=1, inplace=True)

df_c.set_index('id', inplace=True)

df_c = df_c.transpose()

myfunc.save_df_to_h5(df_c, "classes_by_id_string", myfunc.H5_DIR)
myfunc.verbose(f"Cantidad de muestras de cáncer agrupado por el nombre.")
myfunc.verbose(df_c.groupby("target").size())

df_c['target'] = df_c['target'].map(myfunc.TARGET_MAPPING)

myfunc.save_df_to_h5(df_c, "classes_by_id", myfunc.H5_DIR)
myfunc.verbose(f"Cantidad de muestras de cáncer agrupado por el identificador.")
myfunc.verbose(df_c.groupby("target").size())


### 2.- Generar fichero de genes

Selecccionar las lineas que nos interesa del fichero de genes.

Aquellas que 'feature_type == "transcript" and chromosome_name != "chrX" and chromosome_name != "chrY" and chromosome_name != "chrM"'

In [ ]:
df_g = pd.DataFrame()

fichero1="gencode.v44.primary_assembly.annotation.gtf"
df_tmp=myfunc.read_gtf_file(fichero1, myfunc.GENCODE_DIR, myfunc.N_ROWS)
myfunc.verbose(f"Shape : (row,cols) -> {df_tmp.shape}")

df_tmp = df_tmp[df_tmp['chromosome_name'].str.startswith('chr')].query('feature_type == "transcript" and chromosome_name != "chrX" and chromosome_name != "chrY" and chromosome_name != "chrM"')

display(df_tmp)


Finalmente se seleccionan las lineas de genes tipo "protein_coding"

In [ ]:

# Función para extraer valores de la columna 'attributes', pero sólo el primero que encuentra.
def extraer_atributo(valor_atributo, atributo):
    match = re.search(rf'{atributo} ["\']?(.*?)[\'"]?(;|$)', valor_atributo)
    if match:
        return match.group(1)
    else:
        return None

# sólo se obtienen los atributos que interesan del fichero
atributos = ['gene_id','transcript_id','gene_type','gene_name','transcript_type']

for atrib1 in atributos:
    df_tmp[atrib1] = df_tmp['attributes'].apply(lambda x: extraer_atributo(x, atrib1))

df_tmp = df_tmp[df_tmp['chromosome_name'].str.startswith('chr')].query('gene_type == "protein_coding"')

display(df_tmp)


Guardar en un fichero los datos que interesan de los genes, como su id y las posiciones que ocupa.

In [ ]:

quitar_columnas = ["annotation_source","feature_type","score","genomic_strand","genomic_phase","attributes"]

df_tmp.drop(quitar_columnas, axis=1, inplace=True)


df_min=pd.DataFrame(df_tmp.groupby(['gene_id','gene_name','chromosome_name'])['genomic_start_location'].min())
df_max=pd.DataFrame(df_tmp.groupby(['gene_id','gene_name','chromosome_name'])['genomic_end_location'].max())

df_tmp1 = df_min.merge(df_max, left_index=True, right_index=True).sort_values(['chromosome_name','genomic_start_location']).reset_index()

df_tmp1 = df_tmp1.rename({'chromosome_name': 'gChr', 'genomic_start_location': 'gStart', 'genomic_end_location': 'gEnd'}, axis='columns')

display(df_tmp1)

myfunc.save_df_to_h5(df_tmp1.sort_values(['gChr','gStart']), 'transcript.protein_coding', myfunc.H5_DIR)


### 3.- Generar fichero con los valores en las posiciones de los genes.

Leer los ficheros preparados de datos y genes.

In [ ]:
df1 = myfunc.read_h5_to_df("files_joined", myfunc.H5_DIR)[["id","Chr","Start","End"]]

df2 = myfunc.read_h5_to_df("transcript.protein_coding", myfunc.H5_DIR)

#  al ser 1-based, se resta uno para comparar con las posiciones de los bines que son 0-based
df2['gStart'] = df2['gStart'] - 1
df2['gEnd'] = df2['gEnd']

display(df1[df1.Start == 0])
display(df2[df2.gene_name == "MKKS"])

Preparar fichero de datos generando columna con posiciones de los bines por cada cromosoma.

A modo informativo se cuenta el número de genes y de genes x cromosoma


In [ ]:
# Función para extraer el número de la cadena y ordenar por el número
def extract_number_bin(s):
    return int(s.replace("bin", ""))

def extract_number_chr(s):
    return int(s.replace("chr", ""))

df1['chr_number'] = df1['Chr'].apply(extract_number_chr)
df1['pos_number'] = (1+df1['Start']/5000).astype("Int64")

print("distinct Chr",df2["gChr"].unique().size)
print("\ndistinct gene_id",df2["gene_id"].unique().size)

print("\nFilas con gen_id duplicado:")
df_repe1=df2[df2["gene_id"].duplicated()].sort_values(by="gene_name")
display(df_repe1)

print("distinct gene_name",df2["gene_name"].unique().size)

print("\nFilas con gen_name duplicado:")
df_repe1=df2[df2["gene_name"].duplicated()].sort_values(by="gene_name")
display(df2[df2.gene_name.isin(df_repe1["gene_name"])])

df2['chr_number'] = df2['gChr'].apply(extract_number_chr)
display(display(df2.sort_values(by=["chr_number","gStart"])))


Buscar cada gen en que bin se encuentra. 

Con la posición inicial y final, se divide por 5k que es la agrupación en el fichero de datos.

In [ ]:

df3 = df2.copy()
df3['gStart_bin'] = 1 + df3['gStart'].astype('int64')//5000
df3['gEnd_bin'] =1 + df3['gEnd'].astype('int64')//5000

# Función para generar la secuencia de números
def generate_sequence(row):
    return list(range(row['gStart_bin'], row['gEnd_bin'] + 1))

# Aplicar la función y crear una nueva columna 'sequence' en el DataFrame
df3['sequence'] = df3.apply(generate_sequence, axis=1)

df3 = df3.sort_values(by=["chr_number","gStart"])
display(df3)

display(df3[df3.gene_id == "ENSG00000188976.11"])

Eliminar duplicados si los hubiera

In [ ]:

columns_to_keep = ['gChr', 'gene_id', 'chr_number','sequence']

# Aplicar explode para desglosar la lista de números en filas separadas
df4 = df3[columns_to_keep].explode('sequence', ignore_index=True)

# Renombrar las columnas del nuevo DataFrame
df4.columns = ['gChr', 'gene_id', 'chr_number', 'pos_number']

display(df4.sort_values(by=["chr_number","pos_number"]))

df4.drop_duplicates(inplace=True)
display(df4.sort_values(by=["chr_number","pos_number"]))


Visualizar el dataframe original con las posiciones para cada cromosoma

In [ ]:

display(df1.sort_values(by=["chr_number","pos_number"]))

Se cruzan el dataset datos y el generado con los genes.

De esta forma se obtiene cada gen que bines abarca.

In [ ]:
df1['chr_number'] = df1['chr_number'].astype('int64')
df1['pos_number'] = df1['pos_number'].astype('int64')
df4['chr_number'] = df4['chr_number'].astype('int64')
df4['pos_number'] = df4['pos_number'].astype('int64')

display(df1[df1.Chr=="chr2"])
display(df4[df4.gChr=="chr2"])

data = pd.merge(df1, df4, on=['chr_number','pos_number'], how='inner') 
result3 = data.sort_values(by=["chr_number","pos_number"])
display(result3)

myfunc.save_df_to_h5(result3, 'bines_transcript_e', myfunc.H5_DIR)


Seleccionar las filas del fichero completo de datos y hacer un merge con el fichero de genes (con la posición del bin en el que está)

In [ ]:
df1 = myfunc.read_h5_to_df("files_joined", myfunc.H5_DIR)
display(result3)

df2 = result3[["Chr","id","gene_id"]]

display(df1)
display(df2)

df3 = df1.merge(df2, on=["Chr","id"], how="inner")
display(df3)


Eliminar columnas innecesarias

In [ ]:
columnas1 = list(df3.columns)
print(columnas1)
columnas1.remove("Start")
columnas1.remove("End")
columnas1.remove("Strand")
columnas1.remove("Length")
columnas1.remove("id")
columnas1.remove("Chr")
print(columnas1)
df4 = df3[columnas1]
display(df4.sort_values(by="gene_id").head(33))
display(df4[df4.gene_id=="ENSG00000188976.11"].sort_values(by="gene_id").head(33))

Calcular la media de las medidas de los bines por gen de cada muestra

In [ ]:

df_min1=pd.DataFrame(df4.groupby(['gene_id']).mean(numeric_only=True))
display(df_min1)


Se traspone el dataframe.

In [ ]:
df_t = df_min1.transpose()

display(df_t)

if myfunc.SHOW_DISPLAYS:
    display(df_t.shape)
    display(df_t.dtypes)


Se eliminan las columnas con varianza cero/valores constantes.

In [ ]:
varianza0 = np.var(df_t, axis=0)
columnas_var0 = varianza0[varianza0 != 0].index
df_t = df_t[columnas_var0]
display(df_t.shape)

Se guarda un ficheros con los genes, otro fichero con los genes mas el nombre del tipo de cáncer y otro con los genes y el identificador del cáncer.

In [ ]:

myfunc.save_df_to_h5(df_t, "rows_transpose_by_gene_id", myfunc.H5_DIR)

df_s = myfunc.read_h5_to_df("classes_by_id_string", myfunc.H5_DIR)

df_new = df_t.merge(df_s, left_index=True, right_index=True)
df_new.index.name = "id"
display(df_new.reset_index())

if myfunc.SHOW_DISPLAYS:
    display(df_new.shape)
    display(df_new.dtypes)

myfunc.save_df_to_h5(df_new, "rows_transpose_by_gene_id_with_target_str", myfunc.H5_DIR)

df_s = myfunc.read_h5_to_df("classes_by_id", myfunc.H5_DIR)
df_new = df_t.merge(df_s, left_index=True, right_index=True)
df_new.index.name = "id"
display(df_new.reset_index())

if myfunc.SHOW_DISPLAYS:
    display(df_new.shape)
    display(df_new.dtypes)

myfunc.save_df_to_h5(df_new, "rows_transpose_by_gene_id_with_target_num", myfunc.H5_DIR)


### 4.- Normalización de los datos

Se normalizan los datos y se guarda el fichero normalizado con el identificador del tipo de cáncer

In [ ]:
from sklearn.preprocessing import RobustScaler

normalizer = RobustScaler(with_centering=True, with_scaling=True)

df_n = pd.DataFrame(normalizer.fit_transform(df_t), columns=df_t.columns, index=df_t.index)

display(df_n)
display(df_n.shape)

df_new = df_n.merge(df_s, left_index=True, right_index=True)

display(df_new.shape)
display(df_new)

myfunc.save_df_to_h5(df_new, "rows_transpose_norm_by_gene_id_with_target_num", myfunc.H5_DIR)


### 5.- Generar los siguientes ficheros con muestras balanceadas.

Leer fichero con los datos a balancear

In [ ]:
df_t = myfunc.read_h5_to_df("rows_transpose_norm_by_gene_id_with_target_num", myfunc.H5_DIR)

#### 5.1.	Clasificación binaria con submuestreo de datos


##### 221 de control + 37 muestras de los 6 tipos de cáncer (total 222 muestras).

El identificador de este conjunto de datos es "bin_s" y sólo contiene muestras reales.

Se crea el fichero con 222 muestras normalizadas para utilizar en la clasificación binaria. Adicionalmente se crea un fichero de test para hacer pruebas de ejecución.

In [ ]:
np.random.seed(42)

display(df_t.groupby("target").size())
display(df_t.shape)
n_target_0 = df_t[df_t.target==0].groupby("target").size()[0]
n_target_1 = df_t.shape[0] - n_target_0

df_concat = pd.DataFrame()

for class1 in myfunc.TARGET_MAPPING.values():
    if class1 == 0:
        # incluir todas las filas del target 0
        df_tmp = df_t[df_t.target==class1].sample(n_target_0, random_state=42)
        df_tmp.target = 0
    else:
        # incluir un numero constante de filas
        df_tmp = df_t[df_t.target==class1].sample(37, random_state=42)

    if df_concat.empty:
        df_concat = df_tmp.copy()
    else:
        df_concat  = pd.concat([df_concat, df_tmp], axis = 0)

display(df_concat.groupby("target").size())

df_concat.loc[df_concat['target'] != 0, 'target'] = 1

myfunc.verbose("Muestras balanceadas entre cancerosas y sanas.")
display(df_concat.groupby("target").size())

display(df_concat)
myfunc.save_df_to_h5(df_concat, "rows_transpose_norm_by_gene_id_with_target_num_bin_s", myfunc.H5_DIR)
myfunc.save_df_to_h5(df_concat, "rows_transpose_norm_by_gene_id_with_target_num_bin_s_test", myfunc.H5_DIR)

#### 5.2.	Clasificación binaria con submuestreo + sobremuestreo



##### 221 de control + 139 muestras sintéticas (total 360 muestras) + 60 muestras de los 6 tipos de cáncer (total 360 muestras)

El identificador de este conjunto de datos es "bin_m" y contiene muestras reales y sintéticas. 

Se crea el fichero con 360 muestras normalizadas para utilizar en la clasificación binaria. Adicionalmente se crea un fichero de test para hacer pruebas de ejecución.

In [ ]:
from imblearn.over_sampling import SMOTE

np.random.seed(42)

df_t2 =df_t.copy()

muestras_por_target=df_t2.groupby(by="target").size()

df_concat = pd.DataFrame()

X = df_t2.drop(columns="target")
y = df_t2["target"]
display(X.shape)
for class1 in myfunc.TARGET_MAPPING.values():

    n_muestras=muestras_por_target[class1]
    print("muestras clase ",class1,":",n_muestras)

    if class1 == 0:
        smote = SMOTE(sampling_strategy={class1: 360}, random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X, y)
        # concatenar con la clase
        df_tmp = pd.concat([X_resampled, y_resampled], axis=1)
        # se conservan las muestras de esa clase
        df_tmp = df_tmp[df_tmp.target==class1]
    else:
        if n_muestras <= 60:
            smote = SMOTE(sampling_strategy={class1: 60}, random_state=42)
            X_resampled, y_resampled = smote.fit_resample(X, y)
            # concatenat con la clase
            df_tmp = pd.concat([X_resampled, y_resampled], axis=1)
            # se conservan las muestras de esa clase
            df_tmp = df_tmp[df_tmp.target==class1]
        else:
            # como hay más muestras, se hace submuestreo
            df_tmp = df_t[df_t.target==class1].sample(60, random_state=42)

    if df_concat.empty:
        df_concat = df_tmp.copy()
    else:
        df_concat  = pd.concat([df_concat, df_tmp], axis = 0)

display(df_concat.groupby("target").size())

df_concat.loc[df_concat['target'] != 0, 'target'] = 1

myfunc.verbose("Muestras balanceadas entre cancerosas y sanas.")
display(df_concat.groupby("target").size())

display(df_concat)
display(df_concat.dtypes)
myfunc.save_df_to_h5(df_concat, "rows_transpose_norm_by_gene_id_with_target_num_bin_m", myfunc.H5_DIR)
myfunc.save_df_to_h5(df_concat, "rows_transpose_norm_by_gene_id_with_target_num_bin_m_test", myfunc.H5_DIR)


#### 5.3.	Clasificación multicáncer con submuestreo de datos


##### 37 muestras de cada tipo, control + 6 tipos cáncer

El identificador de este conjunto de datos es "mul_s" y solo contiene muestras reales. 

Se crea el fichero con 259 muestras normalizadas para utilizar en la clasificación multicáncer. Adicionalmente se crea un fichero de test para hacer pruebas de ejecución.

In [ ]:
np.random.seed(42)

df_t2 = df_t.copy()
display(df_t2.groupby("target").size())
display(df_t2.shape)

df_concat = pd.DataFrame()

for class1 in myfunc.TARGET_MAPPING.values():
    df_tmp = df_t2[df_t2.target==class1].sample(37, random_state=42)

    if df_concat.empty:
        df_concat = df_tmp.copy()
    else:
        df_concat  = pd.concat([df_concat, df_tmp], axis = 0)

myfunc.verbose("Muestras balanceadas por tipo de cáncer.")
display(df_concat.groupby("target").size())

display(df_concat)
myfunc.save_df_to_h5(df_concat, "rows_transpose_norm_by_gene_id_with_target_num_mul_s", myfunc.H5_DIR)
myfunc.save_df_to_h5(df_concat, "rows_transpose_norm_by_gene_id_with_target_num_mul_s_test", myfunc.H5_DIR)

#### 5.4. Clasificación multicáncer con submuestreo + sobremuestreo


##### 62 muestras de cada tipo, control + 6 tipos cáncer

El identificador de este conjunto de datos es "mul_m" y contiene muestras reales y sintéticas. 

Se crea el fichero con 434 muestras normalizadas para utilizar en la clasificación multicáncer. Adicionalmente se crea un fichero de test para hacer pruebas de ejecución.

In [ ]:
from imblearn.over_sampling import SMOTE

np.random.seed(42)

df_t2 = df_t.copy()
display(df_t2.groupby("target").size())

muestras_por_target=df_t2.groupby(by="target").size()

df_concat = pd.DataFrame()

X = df_t2.drop(columns="target")
y = df_t2["target"]
for class1 in myfunc.TARGET_MAPPING.values():

    n_muestras=muestras_por_target[class1]
    print("muestras clase ",class1,":",n_muestras)

    if class1 == 0:
        df_tmp = df_t2[df_t2.target==class1].sample(62, random_state=42)
    else:
        if n_muestras <= 62:
            smote = SMOTE(sampling_strategy={class1: 62}, random_state=42)
            X_resampled, y_resampled = smote.fit_resample(X, y)
            # concatenar con la clase
            df_tmp = pd.concat([X_resampled, y_resampled], axis=1)
            # se conservan las muestras de esa clase
            df_tmp = df_tmp[df_tmp.target==class1]
        else:
            # como hay más muestras, se hace submuestreo
            df_tmp = df_t2[df_t2.target==class1].sample(62, random_state=42)

    if df_concat.empty:
        df_concat = df_tmp.copy()
    else:
        df_concat  = pd.concat([df_concat, df_tmp], axis = 0)

myfunc.verbose("Muestras balanceadas por tipo de cáncer.")
display(df_concat.groupby("target").size())

display(df_concat)
myfunc.save_df_to_h5(df_concat, "rows_transpose_norm_by_gene_id_with_target_num_mul_m", myfunc.H5_DIR)
myfunc.save_df_to_h5(df_concat, "rows_transpose_norm_by_gene_id_with_target_num_mul_m_test", myfunc.H5_DIR)

El resto de muestras quedan en este fichero serán solo las de tipo 0 y tipo 6 que no han sido vistas por ningún modelo.

In [ ]:
np.random.seed(42)
df_resto = myfunc.read_h5_to_df("rows_transpose_norm_by_gene_id_with_target_num", myfunc.H5_DIR)
display(df_resto.shape)

muestras_por_target=df_resto.groupby(by="target").size()
for class1 in myfunc.TARGET_MAPPING.values():

    n_muestras=muestras_por_target[class1]

    if n_muestras <= 62:
        df_tmp = df_resto[df_resto.target==class1].sample(n_muestras, random_state=42)
        print(f"quedan muestras clase {class1}: {n_muestras} - {df_tmp.shape[0]} = {n_muestras - df_tmp.shape[0]}")
    else:
        df_tmp = df_resto[df_resto.target==class1].sample(62, random_state=42)
        print(f"quedan muestras clase {class1}: {n_muestras} - {df_tmp.shape[0]} = {n_muestras - df_tmp.shape[0]}")

    #  resto de muestras
    df_resto = df_resto[~df_resto.index.isin(df_tmp.index)]

display(df_resto.groupby("target").size())
myfunc.save_df_to_h5(df_resto, "rows_transpose_norm_by_gene_id_with_target_num_resto", myfunc.H5_DIR)

#### 5.5.	Clasificación multicáncer con todas las muestras

##### todas las muestras de cada tipo


In [ ]:
np.random.seed(42)

df_t2 = df_t.copy()
display(df_t2.groupby("target").size())
display(df_t2.shape)


display(df_t2)
myfunc.save_df_to_h5(df_t2, "rows_transpose_norm_by_gene_id_with_target_num_ori_w", myfunc.H5_DIR)
myfunc.save_df_to_h5(df_t2, "rows_transpose_norm_by_gene_id_with_target_num_ori_w_test", myfunc.H5_DIR)

### Finalización del playbook

In [ ]:
end_time = datetime.datetime.now()
total_time = end_time - start_time
myfunc.verbose(f"Notebook ha tardado {total_time} segundos")